In [1]:
from tqdm import tqdm
import datetime as dt
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\sicb.xlsx')

In [3]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [ШК], [Код склада], [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление], 
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток], [Цена Розн., за шт],
[Чистый вес], [Общий вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], 
[Дата закупки], [Код поставщика], [Наименование поставщика]
from [Staging].[Reports].[Remainings_8h]
where [Товарная группа] in (N'СИ БРАСЛЕТ БК', N'СИ ЦЕПЬ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [4]:
df['ID Сайта'] = df['ID Сайта'].astype(str)

In [5]:
df = df[['ID Сайта', 'Цена за грамм расчёт по модели']]

In [6]:
stock = stock.merge(df, how='left', on='ID Сайта')

In [7]:
stock = stock[['ШК', 'Код склада', 'Код товара', 'Артикул поставщика',
    'Внутренний артикул', 'ID Сайта', 'Товарное направление',
    'Товарная группа', 'Товарный кластер', 'Проба', 'Размер изделия',
    'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток', 'Чистый вес', 
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Размер изделия',
    'Цена за грамм расчёт по модели']]

In [8]:
stock['Себес за грамм было'] = stock['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] / stock['Чистый вес']

In [10]:
stock['Наценка после перерасчёта'] = stock['Цена за грамм расчёт по модели'] / stock['Себес за грамм было'] - 1

In [11]:
stock.to_excel('цена за грамм после применения модельной цены.xlsx', index=False)

In [4]:
# discounts = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\Артикул сайта-скидки по ПК 2024.07.12.xlsx')
# discounts = discounts[['Артикул сайта', 'discount_value_promotions_1']]

# def discount_index(discount):
#     try:
#         return 1 - discount / 100
#     except:
#         return 1
    
# discounts.discount_value_promotions_1 = discounts.discount_value_promotions_1.apply(discount_index)

# discounts.rename(columns={'Артикул сайта': 'ID Сайта', 
#                         'discount_value_promotions_1': 'discounts'},
#                 inplace=True)

# stock = stock[stock['ID Сайта'] != '']
# stock['ID Сайта'] = stock['ID Сайта'].astype(int)

# stock = stock.merge(discounts, how='left', on='ID Сайта')